In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [4]:
# Carregar os dados de lap_times e outros arquivos relevantes
lap_times = pd.read_csv('lap_times.csv')
qualifying = pd.read_csv('qualifying.csv')
races = pd.read_csv('races.csv')

# Merge com outros datasets para criar variáveis explicativas
data = lap_times.merge(qualifying, on=['raceId', 'driverId']).merge(races, on='raceId')

# Reduzir o tamanho dos dados para uma amostra menor
data_sample = data.sample(n=10000, random_state=42)  # Exemplo: 10.000 amostras

# Escolher features (número da volta e posição de qualificação)
X_sample = data_sample[['lap', 'position_x']]
y_sample = data_sample['milliseconds']

# Treinar o modelo com os dados reduzidos
model = LinearRegression()
model.fit(X_sample, y_sample)

# Previsões com o conjunto reduzido
predictions_sample = model.predict(X_sample)

# Comparar previsões com valores reais
mae = mean_absolute_error(y_sample, predictions_sample)
mse = mean_squared_error(y_sample, predictions_sample)
rmse = mse ** 0.5

# Exibir resultados
print("Previsões:", predictions_sample)
print("Erro Médio Absoluto (MAE):", mae)
print("Erro Quadrático Médio (MSE):", mse)
print("Raiz do Erro Quadrático Médio (RMSE):", rmse)


Previsões: [ 96830.71810161  90886.61483984 105394.37864319 ...  91282.6623153
 100033.62787001  91150.64649015]
Erro Médio Absoluto (MAE): 15561.3873326806
Erro Quadrático Médio (MSE): 15003154455.928654
Raiz do Erro Quadrático Médio (RMSE): 122487.36447458022


In [5]:
# Carregar os dados
lap_times = pd.read_csv('lap_times.csv')
results = pd.read_csv('results.csv')
qualifying = pd.read_csv('qualifying.csv')
pit_stops = pd.read_csv('pit_stops.csv')

# Substituir valores '\N' por NaN para tratar valores ausentes
lap_times.replace('\\N', np.nan, inplace=True)
results.replace('\\N', np.nan, inplace=True)
qualifying.replace('\\N', np.nan, inplace=True)
pit_stops.replace('\\N', np.nan, inplace=True)

# 1. Preparar os dados
# Média do tempo de volta por piloto e corrida
lap_times_filtered = lap_times.groupby(['raceId', 'driverId'])['milliseconds'].mean().reset_index()
# Selecionar posições de qualificação e filtrar
qualifying_filtered = qualifying[['raceId', 'driverId', 'position']].rename(columns={'position': 'qualifying_position'})
# Filtrar resultados e posições finais
results_filtered = results[['raceId', 'driverId', 'position']].rename(columns={'position': 'final_position'})
# Número de pit stops por corrida e piloto
pit_stops_filtered = pit_stops.groupby(['raceId', 'driverId'])['stop'].count().reset_index()

# Combinar os dados em um único dataframe
data_combined = pd.merge(lap_times_filtered, qualifying_filtered, on=['raceId', 'driverId'])
data_combined = pd.merge(data_combined, results_filtered, on=['raceId', 'driverId'])
data_combined = pd.merge(data_combined, pit_stops_filtered, on=['raceId', 'driverId'])

# Tratar valores ausentes (por exemplo, removendo linhas com NaN)
data_combined.dropna(inplace=True)

# Converter colunas para tipo numérico (caso ainda contenham strings como resultado da importação)
data_combined['milliseconds'] = pd.to_numeric(data_combined['milliseconds'])
data_combined['qualifying_position'] = pd.to_numeric(data_combined['qualifying_position'])
data_combined['final_position'] = pd.to_numeric(data_combined['final_position'])
data_combined['stop'] = pd.to_numeric(data_combined['stop'])

# 2. Pré-processamento
# Variáveis preditoras e variável alvo
X = data_combined[['milliseconds', 'qualifying_position', 'stop']]
y = data_combined['final_position']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Treinar o modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 4. Fazer previsões e avaliar o modelo
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

# Exibir o erro médio absoluto
print(f"Erro Médio Absoluto (MAE): {mae:.2f}")

# Exibir as primeiras previsões e os valores reais
print("\nPrimeiras previsões vs valores reais:")
for pred, real in zip(y_pred[:5], y_test[:5]):
    print(f"Previsto: {int(pred)}, Real: {real}")


Erro Médio Absoluto (MAE): 2.75

Primeiras previsões vs valores reais:
Previsto: 9, Real: 5
Previsto: 11, Real: 2
Previsto: 6, Real: 3
Previsto: 11, Real: 12
Previsto: 6, Real: 17
